In [120]:
from pymongo import MongoClient
import requests
import json
import pandas as pd
import numpy as np
from getpass import getpass
import folium
from folium import Choropleth, Circle, Marker, Icon, Map
from folium.plugins import HeatMap, MarkerCluster

In [121]:
def access_mdb_local_collection (database, collection):

    """ Function that returns a collection inside a DataBase 
    host in my local Server """

    # 1. Connect to my local Server
    client = MongoClient("localhost:27017")
    
    # 2. Access a DataBase:
    db = client[database]

    # 3. Acces a collection inside the given DataBase
    c = db.get_collection(collection)

    # 4. Return collection into a given DataBase host in a given Server
    return c

### Functions:

In [122]:
def extract_parameters(i):
            
            name = i["name"]
            
            lat = i["geocodes"]["main"]["latitude"]
            lon = i["geocodes"]["main"]["longitude"]
            
            #can have more than one:
            cat_list = [x["id"] for x in i["categories"]]
            
            try:
                city = i["location"]["locality"]
            except KeyError:
                city = np.nan
            
            params = {"city":city, "name": name, "cat_code":cat_list, "lat":lat, "lon":lon}
            
            return params

In [123]:
def city_places(city):
    
    "Returs a dataframe with all palces inside a city from our DB"
    
    c = access_mdb_local_collection ("ironhack", "foursquare")
    
    # 1. Set the filter to the objects
    filt = {"location.locality":city}
    
    # 2. Select the filds that we want to target:
    projection = { "_id":0,"categories":1, "geocodes":1,"name":1, "location":1}
    
    # 3. Extract data
    datos = list(c.find(filt,projection))
    
    # 4. Generate an empty dictionary with the categories to save:
    register = {"city": [], "place_name":[], "categories":[],"lat":[], "lon":[]}
    
    # 5. Iterate trowh all elements found:
    for i in datos:

        info = extract_parameters(i) # info it's a dictionary 
        
        register["city"].append(info["city"])
        register["place_name"].append(info["name"])
        register["categories"].append(info["cat_code"])
        register["lat"].append(info["lat"])
        register["lon"].append(info["lon"])

    df = pd.DataFrame(register)
    
    return df

In [124]:
def icon_set(category_code, tags): # marker of categorye [10051]
    
    icon =  Icon(
            color = tags[category_code][1], #red
            icon_color = tags[category_code][2], #black
            icon = tags[category_code][3], #dog
            prefix="fa" # font-awesome website: fa
                )
            
    return icon

In [133]:
def city_markers(df, to_map, tags):
    
        for index, row in df.iterrows():
            
            
            # What if there is no categorie?
    
            if df.iloc[index]["categories"] != []: #--> If it's not empty - take the list 
                
                cat_list = df.iloc[index]["categories"] 
                
                 # What category select? ?
                    
                for key in tags: # --> tags = the ones we have defined
                    if key in cat_list:    
                        category_code = key  # --> we kep the category 
                    else:
                        continue # --> There is no a matching category --> skip  
            else: 
                continue # --> There is no category --> skip 

           
            
            lable =  df.iloc[index]["place_name"]
            
            """ Error that i dont understand -- Investigate """
            
            try:
                #Set the icon for that kind of place:
                icon_conf = icon_set(category_code, tags)
            except UnboundLocalError:
                continue
            
            #Extract position in witch locates:
            lat = df.iloc[index]["lat"]
            lon = df.iloc[index]["lon"]
            coordenates = [lat, lon]
            
            #Set the marker:
            marker = Marker(coordenates, tooltip=lable ,  icon=icon_conf)
            
            marker.add_to(to_map)
        
        return

In [134]:
def heat_city(df, to_map):
    
    HeatMap(data = df[["lat", "lon"]]).add_to(to_map)
        
    return

# Creamos un mapa de tags

1. Coordenates for each city:

In [141]:
locations = {
    "Madrid" : [40.40841191,-3.68760088],
    "Valencia" : [39.47534441,-0.37565717],
    "Oviedo" : [43.3622115,-5.8490144]  
}

2. Set the markers specifications

In [142]:
markers = {
    10051: ["stadium" , "orange", "black", "baseball"],
    16033: ["dog_park", "green", "white", "dog"],
    13377: ["vegan_rest", "white", "green", "leaf"],
}

### MAIN:

In [143]:
#CHANGE THIS VARIABLE TO MAKE THE MAPS:
city = "Oviedo"

1.Create df and map for the city:

In [144]:
df = city_places(city)
city_map =  Map(locations[city], zoom_start=13,  tiles='CartoDB positron')

2. Add tags to the map and save

In [145]:
city_markers(df, city_map, markers)
city_map.save(f"{city}_tags_map.html")

### DATA DRIVEN DECITIONS


    - Oviedo - No vegan enough 

    - Valencia vs Madrid -- meet indispensables - precio medio/metro cuadrado:

        Madrid --> 3852 €/m2

        Valencia --> 1627 €/m2

Fuente: https://www.fotocasa.es/fotocasa-life/compraventa/conoce-el-precio-de-la-vivienda-en-venta-este-mes/